# Simulations

In this notebook are shown the implementation of the Swing algorithm on each one of the different topologies. The simulations returns the congestion $\Xi$ of each topology, and the number of clock cycle for the first phase of the **bandwidth-optimal** version.

We will start by importing all the needed functions from the file **utils.py** and from the library **numpy**.

In [101]:
from utils import *
from numpy import log2

The number of collectives in a node can be set here (or later on).

Be aware of the fact that the simulation is **not** guaranteed to work of non-power-of-two numbers: the exception handling has not been addressed in this work.

In [102]:
n = 1024

# 1D torus

To build the 1-dimensional torus, I use the function wrote to build the *supertorus*, and I set the parameter `k=2`: this way, the communication is guaranteed on private links for steps 0 and 1 (like in the 1D torus).

Each iteration starts with an allocation phase: the queues of each directed edge are built, by enqueueing the number of items for the steps. The item in question is the target node: by doing this, we can compare the item to the node it is in, and thus check whether the packet has reached its destination or not.

The congestion on each edge is evaluated through the step execution: if a node passes on an edge, the occupation number for that edge (not considering the direction).

In [103]:
G = supertorus(n,2)
t = 0
occupation = list()

for s in range(int(log2(n))):

    # allocating to each directed edge their packets
    els = int(n*2**(-s-1)) # the size is halved at every step
    for node in G.nodes:
        trg = target(node, s, n) # finding the target of node at step s, in a collective of n nodes
        if (s%2 + node%2)%2 == 0:
            # at even steps, even nodes communicate clockwise
            # likewise do odd nodes at odd steps
            G.queue[(node, (node+1)%n)].ensqueue([trg]*els)
        else:
            G.queue[(node, (node-1)%n)].ensqueue([trg]*els)

    # creating the dictionary that keeps track of occupation of a link
    occupation.append(dict())
    for edge in G.queue.keys():
        start, end = edge
        if edge not in occupation[s].keys() and (end, start) not in occupation[s].keys():
            occupation[s][(start,end)] = 0

    # sending the packets
    while not G.empty_queues():
        
        t += 1
        for edge in G.queue.keys():
            if not G.queue[edge].is_empty():
                start, end = edge
                trg = G.queue[edge].dequeue()

                # if the packet has not yet arrived, it will be enqueued onto the next edge
                if end != trg:
                    G.queue[(end, (2*end-start)%n)].enqueue(trg)

                # updating the occupation
                try:
                    occupation[s][edge] += 1
                except:
                    occupation[s][(end,start)] += 1


To obtain the final congestion deficiency $\Xi$, the maximum occupation number for each step is considered: they are all summed, and the results is then divided for twice the size of the original vector. The reason for the division is that we are considering messages sent both by the "sending" and the "receiving" node of a couple: for the congestion evaluation, we consider the data sent as a single packet.

In [104]:
cong = sum([max(o.values()) for o in occupation])/(2*n)
print(f'The congestion deficiency, evaluated on a 1D torus of {n} nodes, is {cong:.2f}')
print(f'The result is consistent with the lower bound if it holds that {cong:.2f} < {log2(n)/3:.2f}')
print(f'The execution required {t} clock cycles.')

The congestion deficiency, evaluated on a 1D torus of 1024 nodes, is 2.22
The result is consistent with the lower bound if it holds that 2.22 < 3.33
The execution required 2275 clock cycles.


# *Supertorus*

The simulation is similar to the previous one. Here, in order to make use of the added links, at every iteration we check whether the target is neighbor to the node the packet is passing through: if that is the case, the packet is rerouted to the longer link.

This code can also be used for topologies with more reserved edges (or less, like a normal 1D torus): we only need to set the right value for `k`.

In [105]:
k = 3

G = supertorus(n, k)
t = 0
occupation = list()

for s in range(int(log2(n))):

    els = int(n*2**(-s-1))
    for node in G.nodes:
        trg = target(node, s, n)
        if (s%2 + node%2)%2 == 0:
            G.queue[(node, (node+1)%n)].ensqueue([trg]*els)
        else:
            G.queue[(node, (node-1)%n)].ensqueue([trg]*els)

    occupation.append(dict())
    for edge in G.queue.keys():
        start, end = edge
        if edge not in occupation[s].keys() and (end, start) not in occupation[s].keys():
            occupation[s][(start,end)] = 0

    while not G.empty_queues():
        
        t += 1
        for edge in G.queue.keys():
            if not G.queue[edge].is_empty():
                start, end = edge
                trg = G.queue[edge].dequeue()
                
                if end != trg and trg not in G.neighbor[start]:
                    G.queue[(end, (2*end-start)%n)].enqueue(trg)
                
                # if we can use a longer link, the packet is rerouted
                if end != trg and trg in G.neighbor[start]:
                    try: occupation[s][(start, trg)] += 1
                    except: occupation[s][(trg, start)] += 1
                # else, it keeps on following the previous path
                else: 
                    try: occupation[s][(start,end)] += 1
                    except: occupation[s][(end,start)] += 1
    


In [106]:
cong = sum([max(o.values()) for o in occupation])/(2*n)
print(f'The congestion deficiency, evaluated on a supertorus of {n} nodes and reserved link up to step {k-1}, is {cong:.2f}')
print(f'The execution required {t} clock cycles.')

The congestion deficiency, evaluated on a supertorus of 1024 nodes and reserved link up to step 2, is 2.01
The execution required 2105 clock cycles.


# 2D torus

This section simulates the execution on a 2D torus based on a square tessellation of the plane. We can change the value `b` of the width of the rectangular in which the torus is inscribed: the height `h` will change accordingly to it, and to the number `n` of nodes in the collective.

When communication over one of the two dimensions is exhausted, all packets are sent through one single dimension: and thus the packet size doubles in those steps.

In [107]:
b = 32
h = n // b

G = recTorus(b,h)
t = 0
occupation = list()
l = n//2 if min(b,h) > 1 else n

# horizontal steps
for s in range(int(log2(b))):

    # the size of the packet doubles if communication over one dimension has been exhausted
    els = int(l*2**(-s-1))
    if s > log2(h) or h == 1: els *= 2
    for node in G.nodes:
        x,y = node
        trg = (target(x,s,b), y)
        if (s%2 + x%2)%2 == 0:
            G.queue[(node, ((x+1)%b, y))].ensqueue([trg]*els)
        else:
            G.queue[(node, ((x-1)%b, y))].ensqueue([trg]*els)

    occupation.append(dict())
    for edge in G.queue.keys():
        start, end = edge
        if edge not in occupation[s].keys() and (end, start) not in occupation[s].keys():
            occupation[s][(start,end)] = 0

    while not G.empty_queues():
        
        t += 1
        for edge in G.queue.keys():
            if not G.queue[edge].is_empty():
                start,end = edge
                trg = G.queue[edge].dequeue()
                if end != trg:
                    G.queue[(end, ((2*end[0] - start[0])%b, end[1]))].enqueue(trg)
                try:
                    occupation[s][edge] += 1
                except:
                    occupation[s][(end, start)] += 1

# vertical steps
for s in range(int(log2(h))):

    i = int(s+log2(b))
    els = int(l*2**(-s-1))
    if s > log2(b) or b == 1: els *= 2
    for node in G.nodes:
        x,y = node
        trg = (x, target(y, s, h))
        if (s%2 + y%2)%2 == 0:
            G.queue[(node, (x, (y+1)%h))].ensqueue([trg]*els)
        else:
            G.queue[(node, (x, (y-1)%h))].ensqueue([trg]*els)

    occupation.append(dict())
    for edge in G.queue.keys():
        start, end = edge
        if edge not in occupation[i].keys() and (end, start) not in occupation[i].keys():
            occupation[i][edge] = 0

    while not G.empty_queues():

        t += 1
        for edge in G.queue.keys():
            if not G.queue[edge].is_empty():
                start,end = edge
                trg = G.queue[edge].dequeue()
                if end != trg:
                    G.queue[(end, (end[0], (2*end[1] - start[1])%h))].enqueue(trg)
                try:
                    occupation[i][edge] += 1
                except:
                    occupation[i][(end, start)] += 1


Note that here the execution is divided between a horizontal and a vertical phase: each of them takes up half the vector.

In [108]:
cong = sum([max(o.values()) for o in occupation])/(2*n)
print(f'The congestion deficiency, evaluated on a 2D torus of {n} nodes and dimension {b}x{h}, is {cong:.2f}')
print(f'The execution required {t} clock cycles.')

The congestion deficiency, evaluated on a 2D torus of 1024 nodes and dimension 32x32, is 1.38
The execution required 1408 clock cycles.


# Honeycomb torus

The Honeycomb rectangular torus is based on a hexagonal tessellation of a plane. It is inscribed in a $b \times h$ rectangle: like for the square-based torus, we can change `b` (and `h` will change accordingly).

A Honeycomb torus has the same number of links as a square-based torus, **only** in the horizontal dimension: the first phase of the execution is therefore the same of a square-based torus.

Instead, in the vertical dimension, the links are half the ones of a square-based torus: we have to take double to steps, alternating between a vertical one, and a horizontal one.

In the vertical pass we are traversing the torus from the upper left corner to the lower right one.

In [109]:
b = 32
h = n // b
G = honeycomb(b,h)
t = 0
occupation = list()

# horizontal steps
for s in range(int(log2(b))):

    els = int(n*2**(-s-1))
    for node in G.nodes:
        x,y = node
        trg = (target(x,s,b), y)
        if (s%2 + x%2)%2 == 0:
            G.queue[(node, ((x+1)%b, y))].ensqueue([trg]*els)
        else:
            G.queue[(node, ((x-1)%b, y))].ensqueue([trg]*els)

    occupation.append(dict())
    for key in G.queue.keys():
        start, end = key
        if (start, end) not in occupation[s].keys() and (end, start) not in occupation[s].keys():
            occupation[s][(start,end)] = 0

    while not G.empty_queues():

        t += 1
        for edge in G.queue.keys():
            if not G.queue[edge].is_empty():
                start, end = edge
                trg = G.queue[edge].dequeue()
                if end != trg:
                    x,y = end
                    G.queue[(end, ((2*x-start[0])%b, y))].enqueue(trg)
                try:
                    occupation[s][edge] += 1
                except:
                    occupation[s][(end,start)] += 1

# vertical steps
for i in range(int(log2(h))):

    s = int(i+log2(b))
    els = int(n*2**(-s-1))
    d = abs(distance(i))

    for node in G.nodes:
        x,y = node
        
        if (i%2 + y%2)%2 == 0:
            trg = ((x+d)%b, target(y, i, h))
        else:
            trg = ((x-d)%b, target(y, i, h))

        # enrouting the packet is a little bit more complex here
        # it is likely that there is a more compact way to express it :)
        if i%2 == 0:
            if y%2 == 0:
                if x%2 == 0: G.queue[node, (x, (y+1)%h)].ensqueue([trg]*els)
                else: G.queue[(node, ((x+1)%b, y))].ensqueue([trg]*els)
            else:
                if x%2 == 0: G.queue[(node, (x, (y-1)%h))].ensqueue([trg]*els)
                else: G.queue[(node, ((x-1)%b, y))].ensqueue([trg]*els)
        else:
            if y%2 == 0:
                if x%2 == 0: G.queue[node, ((x-1)%b, y)].ensqueue([trg]*els)
                else: G.queue[(node, (x, (y-1)%h))].ensqueue([trg]*els)
            else:
                if x%2 == 0: G.queue[(node, ((x+1)%b, y))].ensqueue([trg]*els)
                else: G.queue[(node, (x, (y+1)%h))].ensqueue([trg]*els)

    occupation.append(dict())
    for key in G.queue.keys():
        start, end = key
        if (start, end) not in occupation[s].keys() and (end, start) not in occupation[s].keys():
            occupation[s][(start,end)] = 0

    while not G.empty_queues():

        t += 1
        for edge in G.queue.keys():
            if not G.queue[edge].is_empty():
                start, end = edge
                trg = G.queue[edge].dequeue()
                if trg != end:
                    sx, sy = start
                    ex, ey = end

                    # the previous step is one to the left
                    if ex - sx == 1 or ex - sx < -1:
                        G.queue[(end, (ex, (ey+1)%h))].enqueue(trg)

                    # the previous step is one to the right
                    elif ex - sx == -1 or ex - sx > 1:
                        G.queue[(end, (ex, (ey-1)%h))].enqueue(trg)

                    # the previous step is a downward one
                    elif ey - sy == 1 or ey - sy < -1:
                        G.queue[(end, ((ex+1)%b, ey))].enqueue(trg)

                    # the previous step is a upward one
                    else:
                        G.queue[(end, ((ex-1)%b, ey))].enqueue(trg)

                try: occupation[s][(start, end)] += 1
                except: occupation[s][(end, start)] += 1 
    

In [110]:
cong = sum([max(o.values()) for o in occupation])/(2*n)
print(f'The congestion deficiency of a Honeycomb torus of {n} nodes and dimension {b}x{h}, is {cong:.2f}')
print(f'The execution required {t} clock cycles.')

The congestion deficiency of a Honeycomb torus of 1024 nodes and dimension 32x32, is 1.46
The execution required 1506 clock cycles.
